# LeNet Computer Vision Model

> https://doi.org/10.1109/5.726791

## Import modules

In [ ]:
import tensorflow
from keras.datasets.mnist import load_data
from keras.models import Sequential
from numpy import ndarray
from typing import Any, List
from keras import layers, losses
from datetime import datetime
from tensorflow.keras.callbacks import TensorBoard
from pathlib import Path

## Download and preparte MNIST dataset

1. As the `LeNet` model expects images to be of size *32 x 32*, all images within the *MNIST* dataset need to be scaled from *28 x 28* to *32 x 32*
2. As *MNIST* images are in grayscale, we want to binarize them between 0 and 1 (white or black) by dividing their color value by 255
3. As *MNIST* images are in grayscale, they do not have the color channel value that is expected by *Keras* `Conv2d` module. In other words, the *MNIST* dataset tensor structure only contains [`batchSize`, `height`, `width`] Thus, we need to add in a fourth dimension to make our tenors look like [`batchSize`, `height`, `width`, `channel`] where `channel` == 1

In [ ]:
imagePadding: List[List[int]] = [[0, 0], [2, 2], [2, 2]]

mnist: tuple[tuple[Any, Any]] = load_data()

xTrain: ndarray = mnist[0][0]
yTrain: ndarray = mnist[0][1]
xTest: ndarray = mnist[1][0]
yTest: ndarray = mnist[1][1]

xTrain = tensorflow.pad(tensor=xTrain, paddings=imagePadding) / 255
xTest = tensorflow.pad(tensor=xTest, paddings=imagePadding) / 255

xTrain = tensorflow.expand_dims(input=xTrain, axis=3)
xTest = tensorflow.expand_dims(input=xTest, axis=3)

## Build the model

### Architecture

[![https://production-media.paperswithcode.com/methods/LeNet_Original_Image_48T74Lc.jpg](https://production-media.paperswithcode.com/methods/LeNet_Original_Image_48T74Lc.jpg)](https://production-media.paperswithcode.com/methods/LeNet_Original_Image_48T74Lc.jpg)

> Image from https://production-media.paperswithcode.com/methods/LeNet_Original_Image_48T74Lc.jpg

In [ ]:
lenet: Sequential = Sequential(name="LeNet")
lenet.add(layer=layers.Conv2D(filters=6, kernel_size=5, activation="tanh"))
lenet.add(layers.AveragePooling2D(pool_size=2))
lenet.add(layer=layers.Activation(activation="sigmoid"))
lenet.add(layers.Conv2D(16, 5, activation="tanh"))
lenet.add(layers.AveragePooling2D(2))
lenet.add(layers.Activation("sigmoid"))
lenet.add(layers.Conv2D(120, 5, activation="tanh"))
lenet.add(layers.Flatten())
lenet.add(layers.Dense(84, activation="tanh"))
lenet.add(layers.Dense(10, activation="softmax"))
lenet.build(input_shape=xTrain.shape)
lenet.compile(
    optimizer="adam",
    loss=losses.sparse_categorical_crossentropy,
    metrics=["accuracy"],
)
lenet.summary()

## Train the model

In [ ]:
logFolder: Path = Path("logs/" + datetime.now().strftime("%Y%m%d-%H%M%S"))

tensorboard_callback: TensorBoard = TensorBoard(
    log_dir=logFolder,
    histogram_freq=1,
    write_images=True,
)

lenet.fit(
    x=xTrain,
    y=yTrain,
    batch_size=64,
    epochs=5,
    callbacks=[tensorboard_callback],
)